In [1]:
import numpy as np
import pandas as pd
from numba import njit

In [2]:
df = pd.read_csv('../data/BTCUSDT_1h_2020-2025.csv')

In [3]:
@njit
def calculate_linear_regression_slopes(prices: np.ndarray, window: int) -> np.ndarray:

    n = len(prices)
    slopes = np.zeros(n)
    time_indices = np.arange(window)
    
    for i in range(window - 1, n):
        price_window = prices[i-window+1:i+1]
        time_mean = (window - 1) / 2
        price_mean = np.mean(price_window)
        
        # Calculate slope using least squares method
        numerator = np.sum((time_indices - time_mean) * (price_window - price_mean))
        denominator = np.sum((time_indices - time_mean) ** 2)
        slopes[i] = numerator / denominator if denominator != 0 else 0
    
    return slopes

def find_regimes(df: pd.DataFrame, regression_window: int = 10) -> pd.DataFrame:

    slopes = calculate_linear_regression_slopes(df['Close'].values, regression_window)
    df['Slope'] = slopes
    
    slope_volatility = pd.Series(slopes).rolling(regression_window).std().values
    df['Regime'] = 'Trend'
    df.loc[abs(df['Slope']) < slope_volatility, 'Regime'] = 'Range'
    
    return df

df_regimes = find_regimes(df, regression_window = 20)

In [4]:
df_regimes.tail(20)

,Timestamp,Open,High,Low,Close,Volume,Slope,Regime
44512,2025-01-28 16:00:00,102730.6,102870.3,101850.0,102211.8,11332.929,54.632256,Trend
44513,2025-01-28 17:00:00,102211.8,103065.9,102170.3,102525.2,9048.729,39.074887,Range
44514,2025-01-28 18:00:00,102525.3,103089.9,102123.9,102161.1,8192.323,20.876692,Range
44515,2025-01-28 19:00:00,102161.0,102624.0,101778.0,102368.7,6720.676,11.665414,Range
44516,2025-01-28 20:00:00,102368.7,102495.0,101195.2,101226.3,8239.896,-17.685113,Range
44517,2025-01-28 21:00:00,101226.3,101489.0,100240.2,100298.7,10624.425,-67.188195,Range
44518,2025-01-28 22:00:00,100298.7,101379.2,100235.0,101220.4,8205.542,-85.882105,Range
44519,2025-01-28 23:00:00,101220.4,101400.0,100700.0,101279.6,4671.100,-98.210301,Range
44520,2025-01-29 00:00:00,101279.7,101770.3,101269.7,101651.6,4797.692,-98.805714,Range
44521,2025-01-29 01:00:00,101651.6,101966.3,101422.2,101806.1,2744.621,-93.728647,Range
